In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e11/sample_submission.csv
/kaggle/input/playground-series-s5e11/train.csv
/kaggle/input/playground-series-s5e11/test.csv


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

In [3]:
# ignore the warnings for clean output
warnings.filterwarnings('ignore')

In [4]:
train_df = pd.read_csv('/kaggle/input/playground-series-s5e11/train.csv')
test_df = pd.read_csv('/kaggle/input/playground-series-s5e11/test.csv')
submission_df = pd.read_csv('/kaggle/input/playground-series-s5e11/sample_submission.csv')

In [9]:
train_df.shape

(593994, 13)

In [11]:
test_df.shape

(254569, 12)

In [14]:
train_df.head()

,id,annual_income,debt_to_income_ratio,credit_score,loan_amount,interest_rate,gender,marital_status,education_level,employment_status,loan_purpose,grade_subgrade,loan_paid_back
0,0,29367.99,0.084,736,2528.42,13.67,Female,Single,High School,Self-employed,Other,C3,1.0
1,1,22108.02,0.166,636,4593.10,12.92,Male,Married,Master's,Employed,Debt consolidation,D3,0.0
2,2,49566.20,0.097,694,17005.15,9.76,Male,Single,High School,Employed,Debt consolidation,C5,1.0
3,3,46858.25,0.065,533,4682.48,16.10,Female,Single,High School,Employed,Debt consolidation,F1,1.0
4,4,25496.70,0.053,665,12184.43,10.21,Male,Married,High School,Employed,Other,D1,1.0


In [16]:
train_df.columns.tolist()

['id',
 'annual_income',
 'debt_to_income_ratio',
 'credit_score',
 'loan_amount',
 'interest_rate',
 'gender',
 'marital_status',
 'education_level',
 'employment_status',
 'loan_purpose',
 'grade_subgrade',
 'loan_paid_back']

In [22]:
target = 'loan_paid_back'
print('Target Distribution:')
train_df[target].value_counts(normalize=True)

Target Distribution:


loan_paid_back
1.0    0.79882
0.0    0.20118
Name: proportion, dtype: float64

In [26]:
print('Missing Values in Train:')
train_df.isnull().sum()

Missing Values in Train:


id                      0
annual_income           0
debt_to_income_ratio    0
credit_score            0
loan_amount             0
interest_rate           0
gender                  0
marital_status          0
education_level         0
employment_status       0
loan_purpose            0
grade_subgrade          0
loan_paid_back          0
dtype: int64

In [27]:
if 'id' in train_df.columns:
    train_df = train_df.drop(columns=['id'])
    test_df = test_df.drop(columns=['id'])

In [33]:
cat_cols = [i for i in train_df.columns if train_df[i].dtype == 'object' and i != target]
num_cols = [i for i in train_df.columns if train_df[i].dtype != 'object' and i != target]

In [50]:
print('Categorical Columns:-', cat_cols, '\n\n', 'Number Columns:-', num_cols)

Categorical Columns:- ['gender', 'marital_status', 'education_level', 'employment_status', 'loan_purpose', 'grade_subgrade'] 

 Number Columns:- ['annual_income', 'debt_to_income_ratio', 'credit_score', 'loan_amount', 'interest_rate']


In [51]:
print('lenght of cat_cols:', len(cat_cols))
print('lenght of num_cols:', len(num_cols))

lenght of cat_cols: 6
lenght of num_cols: 5


In [52]:
for col in num_cols:
    median_value = train_df[col].median()
    train_df[col] = train_df[col].fillna(median_value)
    test_df[col] = test_df[col].fillna(median_value) # To prevent from the data leakage we won't pass the test's median

In [53]:
for col in cat_cols:
    mode_value = train_df[col].mode()
    train_df[col] = train_df[col].fillna(mode_value)
    test_df[col] = test_df[col].fillna(mode_value)

In [55]:
from sklearn.preprocessing import LabelEncoder

In [58]:
for col in cat_cols:
    le = LabelEncoder()
    combined_data = pd.concat([train_df[col], test_df[col]], axis=0).astype(str)
    le.fit(combined_data)
    train_df[col] = le.transform(train_df[col].astype(str))
    test_df[col] = le.transform(test_df[col].astype(str))

train_df.head()

,annual_income,debt_to_income_ratio,credit_score,loan_amount,interest_rate,gender,marital_status,education_level,employment_status,loan_purpose,grade_subgrade,loan_paid_back
0,29367.99,0.084,736,2528.42,13.67,0,2,1,2,6,4,1.0
1,22108.02,0.166,636,4593.10,12.92,1,1,2,0,2,9,0.0
2,49566.20,0.097,694,17005.15,9.76,1,2,1,0,2,6,1.0
3,46858.25,0.065,533,4682.48,16.10,0,2,1,0,2,18,1.0
4,25496.70,0.053,665,12184.43,10.21,1,1,1,0,6,7,1.0
